##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [1]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizing from 0 - 255 to 0 - 1
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [6]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [21]:
# Sequential model

# The first layer in Sequential always needs the input_shape as a kwarg (no matter if it starts with Flatten, Dense etc)
model = tf.keras.models.Sequential([
                                    
  # Like x.view(x.size(0), -1) # New shape: [batch_size, 28*28=784]
  # This is why Torch is more pythonic  --  Here the batch size is treated "under the hood"               
  tf.keras.layers.Flatten(input_shape=(28, 28)),

  # > Applies activation(dot(input, kernel) + bias)
  # > Kernel is the weights matrix created
  # > Units (128) will be the dimension of the output
    # Essentially it creates a kernel matrix of shape (28, 128)
    # The dot product of (28, 28) & (28, 128) = (28, 128)
    # >> But since we have flatten before, it will actually be:
    # (28*28, ) 
    # > dot((28*28, 1), (1, 128)) = (28*28, 128)
    # With BS: dot((BS, 28*28), (28*28, 128) = (BS, 128)
  # > In Torch we need to put in nn.Linear(28*28, 128), in TF the 28*28 are calculated for us based on the input_shape provided at beg.
  tf.keras.layers.Dense(128, activation='relu'),

  tf.keras.layers.Dropout(0.2),

  # dot((BS, 128), (128, 10)) = (BS, 10) > For all 10 numbers
  # Here no function is specified so there is no non-linearity; just a linear layer
  tf.keras.layers.Dense(10)
])



For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [9]:
# First example
print(x_train[:1].shape)

(1, 28, 28)


In [7]:
predictions = model(x_train[:1]).numpy()
# Probabilities for 10 numbers - in This case predicts the last one
predictions

array([[-0.3966302 , -0.06659387, -0.0873878 , -0.3497427 , -0.18486574,
        -0.20841378, -0.08401708,  0.21445   , -0.11779967,  0.2298936 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [10]:
# Split from main Sequential ( or apply log to it like log_softmax in Torch, to make softmax more numerically stable)
tf.nn.softmax(predictions).numpy()

array([[0.07327908, 0.10193258, 0.09983487, 0.07679678, 0.09056245,
        0.0884548 , 0.10017196, 0.13501088, 0.09684442, 0.13711213]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [12]:
# Same as nn.CrossEntropyLoss in torch
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [13]:
# labels ; predictions
loss_fn(y_train[:1], predictions).numpy()

2.4252636

In [22]:
# Configures model for training
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [24]:
# inputs; labels
# Instead of training loop in pytorch
# Batch size defaults to 32
model.fit(x_train, y_train, batch_size=None, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0804 - accuracy: 0.9753
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0688 - accuracy: 0.9780
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0611 - accuracy: 0.9804
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0546 - accuracy: 0.9822
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0503 - accuracy: 0.9834


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [25]:
# >> W/o performing backwards pass
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0753 - accuracy: 0.9776


[0.07531102001667023, 0.9775999784469604]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [26]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [27]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.9119021e-08, 4.6594288e-09, 8.3193873e-07, 1.0190512e-04,
        4.7541849e-12, 1.0144830e-10, 9.4341640e-16, 9.9988830e-01,
        9.0297263e-08, 8.7928629e-06],
       [1.3412559e-13, 3.9896654e-06, 9.9999571e-01, 3.3011028e-07,
        6.0414356e-21, 8.0193541e-10, 1.2725191e-12, 2.4445998e-17,
        2.5080984e-11, 6.2618908e-18],
       [6.7563688e-10, 9.9963629e-01, 3.7676400e-06, 2.7457818e-07,
        2.5100346e-05, 4.9743067e-09, 2.6089828e-07, 2.3628341e-04,
        9.8111894e-05, 1.8972075e-08],
       [9.9976844e-01, 7.3491586e-12, 3.3664173e-06, 1.3718793e-09,
        5.0467708e-08, 3.4769773e-08, 2.2533823e-04, 1.1858805e-06,
        4.2502020e-09, 1.6865469e-06],
       [8.6127230e-08, 2.3323869e-11, 5.0500256e-07, 8.8563318e-10,
        9.9682415e-01, 4.3083842e-10, 4.8500031e-09, 4.0404611e-06,
        3.3153619e-08, 3.1712207e-03]], dtype=float32)>

In [28]:
probability_model(x_test[:5]).shape

TensorShape([5, 10])